In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 12 18:20:37 2022

@author: 18053
"""


import os
import pandas as pd
import numpy as np
import copy

df = pd.read_csv('data.tsv', sep='\t', header=None)
df.columns = ['word', 'relation', 'related_word']
df = df[df['relation'] == "rel:etymology"]

df['word'] = df['word'].apply(lambda x: x.split(': '))
df['related_word'] = df['related_word'].apply(lambda x: x.split(': '))

df['word_language'] = df['word'].apply(lambda x: x[0])
df['word'] = df['word'].apply(lambda x: x[1])

df['related_word_language'] = df['related_word'].apply(lambda x: x[0])
df['related_word'] = df['related_word'].apply(lambda x: x[1])

df = df[['word', 'word_language', 'related_word',
         'related_word_language']]

#df = df[~df['word'].str.contains('-')]

x = df[(df['word_language'] == 'eng')]
pivot = pd.pivot_table(data = x, index = 'related_word_language',
                       values='word_language',
                       aggfunc='count').sort_values('word_language', ascending=False)
# enm = middle english
# ang = old english
print(pivot.head(15))

In [ ]:
df.to_csv('small_data.csv')

In [ ]:
copy_df = copy.deepcopy(df)
copy_df.columns = ['related_word', 'related_word_language', 'word3', 'word3_language']

In [ ]:
new_df = df.merge(copy_df, on = ['related_word', 'related_word_language'], how='inner')
new_df.head()

In [ ]:
def find_root_language(word):
    print(word)
    temp = df[(df['word'] == word) & 
              ~(df['related_word'].str.contains('-')) &
               (df['word_language'] == 'eng')]
    #print(temp)
    if temp.shape[0] == 0:
        temp = df[(df['word'] == word) & (df['word_language'] == 'eng')]
        if temp.shape[0] == 0:
            return np.nan
        
    new_word, language = temp.iloc[0, 2], temp.iloc[0, 3]
    if new_word == word:
        return np.nan
    
    if language == 'eng':
        return find_root_language(new_word)
    else:
        return language

In [ ]:
english_df = df[df['word_language'] == 'eng']

In [ ]:
test_df = copy.deepcopy(english_df)

In [ ]:
english_df = english_df[~english_df['word'].str.contains('\'')]
english_df = english_df[~english_df['word'].str.contains('-')]
english_df = english_df[~english_df['word'].str.contains('\d')]
english_df = english_df[english_df['word'] <= 'zzzzzz']
english_df['word'] = english_df['word'].apply(lambda x: x.lower())

In [ ]:
word_li = english_df['word'].values

In [ ]:
english_df['origin'] = english_df['word'].apply(lambda x: word_dict[x])

In [ ]:
english_df[~pd.notnull(english_df['origin'])].tail()

In [ ]:
def find_root_language(word):
    temp = df[(df['word'] == word) &
              ~(df['related_word'].str.contains('-')) &
               (df['word_language'] == 'eng')]

    if temp.shape[0] == 0:
        temp = df[(df['word'] == word) & (df['word_language'] == 'eng')]
        if temp.shape[0] == 0:
            return None
       
    new_word, language = temp.iloc[0, 2], temp.iloc[0, 3]
    
    if new_word == word and language == 'eng':
        return None
    
    if language == 'eng':
        return find_root_language(new_word)
    else:
        return language

In [ ]:
import time

In [ ]:
i = 0
j = 0
for word in word_dict.keys():
    if word_dict[word] == None:
        new_lang = find_root_language(word)
        word_dict[word] = new_lang
        
        if new_lang != None:
            if i % 100 == 0 and i > 0:
                t0 = time.perf_counter()
                print('outputting to json ending with word ', word, ' number', i)
                with open("sample.json", "w") as outfile:
                    json.dump(word_dict, outfile)
                t1 = time.perf_counter()
                print(f"Outputted json in {t1 - t0:0.2f} seconds")
            i += 1
    j += 1
    if j % 1000 == 0:
            print('current word:', word, ' number:', j)

In [ ]:
english_df['origin'] = english_df['word'].apply(lambda x: word_dict[x])

In [ ]:
final_df = copy.deepcopy(english_df)
final_df = final_df.drop_duplicates('word',keep='first')

In [ ]:
codes = pd.read_csv('language-codes-full_csv.csv')
#codes.columns = ['origin', 'two', 'full_name']
codes = codes.iloc[:, [0,3]]
codes.columns = ['origin', 'full_name']

In [ ]:
final_df = final_df.merge(codes, on='origin', how='left')

In [ ]:
final_df[pd.notnull(final_df['full_name'])]

In [ ]:
pivot = pd.pivot_table(data=final_df[pd.notnull(final_df['full_name'])],
        index=['full_name'],
        values=['word'],
        aggfunc='count').reset_index()

In [ ]:
pivot.to_csv('language_list.csv')

In [ ]:
grouping = pd.read_csv('test.csv')
grouping.head()

In [ ]:
final_df = final_df.merge(grouping, on='full_name', how='left')

In [ ]:
language_df = final_df.loc[pd.notnull(final_df['origin']), ['word', 'origin', 'full_name', 'Group']]

In [ ]:
pivot = pd.pivot_table(data=language_df[pd.isnull(language_df['full_name'])],
               index='origin',
               values=['word'],
               aggfunc='count').reset_index().sort_values('word', ascending=False)

In [ ]:
pivot[pivot['word'] > 10]

In [ ]:
language_df.loc[language_df['origin'] == 'fra','full_name'] = 'French'
language_df.loc[language_df['origin'] == 'fra','Group'] = 'Latin'

In [ ]:
language_df.loc[language_df['origin'] == 'deu','full_name'] = 'German'
language_df.loc[language_df['origin'] == 'deu','Group'] = 'Germanic'

In [ ]:
language_df.loc[language_df['origin'] == 'nld','full_name'] = 'Dutch;Flemish'
language_df.loc[language_df['origin'] == 'nld','Group'] = 'Germanic'

In [ ]:
language_df.loc[language_df['origin'] == 'xno','full_name'] = 'Anglo-Norman'
language_df.loc[language_df['origin'] == 'xno','Group'] = 'Germanic'

In [ ]:
language_df.loc[language_df['origin'] == 'msa','full_name'] = 'Malay'
language_df.loc[language_df['origin'] == 'msa','Group'] = 'Other'

In [ ]:
language_df.loc[language_df['origin'] == 'fas','full_name'] = 'Persian'
language_df.loc[language_df['origin'] == 'fas','Group'] = 'Other'

In [ ]:
language_df.loc[language_df['origin'] == 'hye','full_name'] = 'Armenian'
language_df.loc[language_df['origin'] == 'hye','Group'] = 'Other'

In [ ]:
language_df.loc[language_df['origin'] == 'cmn','full_name'] = 'Mandarin Chinese'
language_df.loc[language_df['origin'] == 'cmn','Group'] = 'Other'

In [ ]:
language_df.loc[language_df['origin'] == 'ell','full_name'] = 'Modern Greek'
language_df.loc[language_df['origin'] == 'ell','Group'] = 'Other'

In [ ]:
language_data = pd.read_csv("language_data.csv")
language_data = language_data[pd.notnull(language_data['full_name'])]
language_data = language_data[pd.notnull(language_data['word'])]
language_data = language_data[pd.notnull(language_data['Group'])]
language_data.loc[language_data['full_name'].str.contains('Greek'), 'Group'] = 'Greek'
language_data.loc[language_data['full_name'].str.contains('Turkish'), 'Group'] = 'Other'
language_data['word_len'] = language_data['word'].apply(lambda x: len(x))
language_data = language_data[language_data['word_len'] < 25]
language_data.to_csv('language_data.csv', index=False)